<a href="https://colab.research.google.com/github/amoyag/Biofisica/blob/student/S3_FFL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Feed Forward Loop (FFL)**

## **Introducción**
*Nota: Es conveniente que amplíes este resumen teórico con los apartados 3.1 a 3.5 de An Introduction to Systems Biology de Uri Alon. Además, puedes ver estos circuitos y los anteriores en [esta review de Alon](https://doi.org/10.1038/nrg2102).*

Los **motivos de red** (network motifs) son patrones recurrentes en redes de regulación génica y señalización celular. Su frecuencia no es casual: confieren ventajas funcionales que han sido seleccionadas evolutivamente. Entre ellos, el **Feed Forward Loop (FFL)** es uno de los más estudiados.

Un FFL está formado por tres nodos:

*   **X** regula **Y**.
*   **X** regula **Z**.
*   **Y** también regula **Z**.

Este motivo aparece en sistemas tan diversos como la respuesta al estrés en bacterias, la regulación del desarrollo en organismos multicelulares y circuitos neuronales. Su importancia radica en que introduce **procesamiento lógico** en la red, permitiendo filtrar señales y generar respuestas temporales específicas.

***

### **Tipos de FFL**

Existen dos categorías principales:

*   **Coherente FFL:** Las dos rutas (X→Z y X→Y→Z) actúan en la misma dirección sobre Z (por ejemplo, ambas activan).
*   **Incoherente FFL:** Las rutas actúan en direcciones opuestas (por ejemplo, X activa Z directamente, pero Y la reprime).

En esta sesión nos centraremos en el **FFL coherente tipo 1 (C1-FFL)**, el más común en redes génicas.

***

### **Lógica de integración: AND vs OR**

En un **C1-FFL**, la activación de Z depende de las señales provenientes de X y Y. Existen dos modos de integración:

*   **Lógica OR:** Z se activa si **X** o **Y** están presentes. Esto permite respuestas rápidas, porque basta con que X se active para inducir Z.
*   **Lógica AND:** Z se activa solo si **X** y **Y** están presentes simultáneamente. Esto introduce un **retardo en la respuesta**, porque Y necesita tiempo para acumularse tras la activación de X.

En esta práctica nos centraremos en la **lógica AND**, porque es la que confiere propiedades más interesantes desde el punto de vista dinámico: filtrado de pulsos y persistencia.

***

### **Modelo Matemático**

Consideramos tres variables:  

$X(t), ; Y(t), ; Z(t)$


Con ecuaciones diferenciales simplificadas:


$$\frac{dX}{dt} = \beta_X \cdot u(t) - \alpha_X X$$


$$\frac{dY}{dt} = \frac{\beta_Y X}{K_Y + X} - \alpha_Y Y$$


$$\frac{dZ}{dt} = \frac{\beta_Z X}{K_Z + X} \cdot \frac{Y}{K'_Z + Y} - \alpha_Z Z$$

Donde:

*   $u(t)$: entrada externa (pulsos o escalones).
*   $\beta_i$: tasa máxima de síntesis.
*   $\alpha_i$: tasa de degradación.
*   $K_i$: constantes de semisaturación.

La multiplicación de los términos en la ecuación de $Z$ refleja la lógica **AND**: $Z$ solo se activa si $X$ y $Y$ están presentes.

***

### **Propiedades Funcionales del C1-FFL**

#### **a) Respuesta retardada**

En lógica AND, Z no responde inmediatamente a X porque depende también de Y, que necesita tiempo para acumularse. Este retardo es útil para evitar respuestas precipitadas ante señales transitorias.

#### **b) Filtro de pulsos**

Si la señal de entrada (u(t)) es un pulso breve, X se activa pero Y no alcanza niveles suficientes para activar Z. El circuito actúa como un **filtro pasa-largos**, ignorando señales fugaces.

#### **c) Persistencia frente a ruido**

El FFL reduce activaciones espurias causadas por fluctuaciones rápidas en X. Solo señales sostenidas activan Z, lo que confiere **robustez temporal**.

#### **d) Ventaja evolutiva**

Este motivo aparece en procesos donde es crítico distinguir entre señales reales y ruido, como la respuesta a estrés o la activación de programas de desarrollo.


## Banco de módulos

In [ ]:
#@title Módulo 0: Librerías y configuración.

# Módulo 0: Librerías y configuración
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import solve_ivp

# Estilo visual consistente con sesión 2
plt.style.use('classic')
sns.set_theme(style="whitegrid")


In [ ]:
#@title Módulo 1: Modelos
def model_ffl_and(t, y, params):
    X, Y, Z = y
    u = params["input"](t)
    beta_X, alpha_X = params["X"]["beta"], params["X"]["alpha"]
    beta_Y, alpha_Y, K_Y = params["Y"]["beta"], params["Y"]["alpha"], params["Y"]["K"]
    beta_Z, alpha_Z, K_Z, Kp_Z = params["Z"]["beta"], params["Z"]["alpha"], params["Z"]["K"], params["Z"]["Kp"]
    nY = params["Z"].get("nY", 1)  # por defecto 1 (sin cooperatividad)

    dXdt = beta_X * u - alpha_X * X
    dYdt = (beta_Y * X / (K_Y + X)) - alpha_Y * Y
    act_X = X / (K_Z + X)
    act_Y = (Y**nY) / (Kp_Z**nY + Y**nY)
    dZdt = beta_Z * act_X * act_Y - alpha_Z * Z
    return [dXdt, dYdt, dZdt]




def model_cascade(t, y, params):
    X, Z = y
    # Call the input function with time t
    u = params["input"](t)
    beta_X, alpha_X = params["X"]["beta"], params["X"]["alpha"]
    beta_Z, alpha_Z, K_Z = params["Z"]["beta"], params["Z"]["alpha"], params["Z"]["K"]

    dXdt = beta_X * u - alpha_X * X
    dZdt = (beta_Z * X / (K_Z + X)) - alpha_Z * Z
    return [dXdt, dZdt]


In [ ]:
#@title Módulo 2: Configuración (build_config)

def build_config(model="ffl", input_type="step", t_span=(0, 100), y0=None):
    if input_type == "step":
        input_func = lambda t: 1 if t >= 0 else 0
    elif input_type == "pulse":
        input_func = lambda t: 1 if 0 <= t <= 20 else 0

    params = {
        "input": input_func,
        "X": {"beta": 1.0, "alpha": 0.1},
        "Y": {"beta": 1.0, "alpha": 0.5, "K": 1.0},
        "Z": {"beta": 1.0, "alpha": 0.05, "K": 1.0, "Kp": 1.0, "nY": 1}
    }

    if y0 is None:
        y0 = [0, 0, 0] if model == "ffl" else [0, 0]

    return params, t_span, y0


In [ ]:
#@title Módulo 3: Ejecución (run_simulation)

def run_simulation(model_func, params, t_span, y0, t_eval=None):
    if t_eval is None:
        t_eval = np.linspace(t_span[0], t_span[1], 1000)
    sol = solve_ivp(model_func, t_span, y0, args=(params,), t_eval=t_eval)
    return sol.t, sol.y


In [ ]:
#@title Módulo 4: Visualización (plot_timeseries, plot_heatmap)

def plot_timeseries(t, y, labels):
    plt.figure(figsize=(8, 5))
    for i, label in enumerate(labels):
        plt.plot(t, y[i], label=label)
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración")
    plt.legend()
    plt.show()

def plot_heatmap(matrix, x_vals, y_vals, xlabel, ylabel):
    plt.figure(figsize=(6, 5))
    plt.imshow(matrix, origin='lower', aspect='auto', extent=[x_vals[0], x_vals[-1], y_vals[0], y_vals[-1]])
    plt.colorbar(label="Tiempo de activación")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()


In [ ]:
#@title Módulo 5: Análisis (compute_activation_time)

def compute_activation_time(t, z, threshold=0.5):
    max_val = np.max(z)
    target = threshold * max_val
    idx = np.where(z >= target)[0]
    return t[idx[0]] if len(idx) > 0 else None


In [ ]:
#@title Módulo 6: Barrido de parámetros (parameter_scan)

# Módulo 6: Barrido de parámetros y visualización con Seaborn
def parameter_scan(model_func, base_params, t_span, y0, param_x, param_y, x_range, y_range):
    """
    Realiza un barrido de parámetros para dos variables y calcula el tiempo de activación de Z.

    Args:
        model_func: función del modelo (FFL AND).
        base_params: diccionario con parámetros base.
        t_span: tupla (t0, tf).
        y0: condiciones iniciales.
        param_x: nombre del parámetro en eje X (ej. 'beta').
        param_y: nombre del parámetro en eje Y (ej. 'alpha').
        x_range: lista o array con valores para param_x.
        y_range: lista o array con valores para param_y.

    Returns:
        matriz con tiempos de activación.
    """
    results = np.zeros((len(y_range), len(x_range)))

    for i, y_val in enumerate(y_range):
        for j, x_val in enumerate(x_range):
            params = base_params.copy()
            params["Y"]["beta"] = x_val
            params["Y"]["alpha"] = y_val

            t, sol_y = run_simulation(model_func, params, t_span, y0)
            activation_time = compute_activation_time(t, sol_y[-1])
            results[i, j] = activation_time if activation_time else np.nan

    return results

def plot_heatmap(matrix, x_vals, y_vals, xlabel, ylabel, cmap="viridis"):
    """
    Genera un mapa de calor con Seaborn para visualizar el tiempo de activación.
    """
    plt.figure(figsize=(8, 6))
    ax = sns.heatmap(matrix, xticklabels=np.round(x_vals, 2), yticklabels=np.round(y_vals, 2),
                     cmap=cmap, annot=False, cbar_kws={'label': 'Tiempo de activación'})
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.title("Mapa de calor: tiempo de activación de Z")
    plt.show()


## Ejercicios



### **Ejercicio 1: Comparación FFL vs Cascada Simple (Interactivo)**

**Objetivo:**  
Explorar cómo el motivo **Feed Forward Loop (FFL)** con lógica AND introduce un **retardo en la activación de Z** en comparación con una **cascada simple (X → Z)**.

**Instrucciones:**

1.  Usa los **sliders interactivos** para modificar los parámetros clave:
    *   **β_X** y **α_X**: controlan la dinámica de X.
    *   **β_Y** y **α_Y**: controlan la dinámica de Y en el FFL.
2.  Observa cómo cambian las curvas temporales de X, Y y Z en ambos circuitos.
3.  Analiza:
    *   ¿Cómo afecta la velocidad de síntesis y degradación de Y al **retardo en Z**?
    *   ¿Por qué el FFL responde más lentamente que la cascada simple?
4.  Interpreta los **tiempos de activación** que se muestran en la salida:
    *   ¿Qué parámetros hacen que el retardo sea más pronunciado?
    *   ¿Qué ocurre si Y se degrada muy rápido?





In [ ]:



#@title Ejercicio 1: Comparación FFL vs Cascada Simple (Interactivo)

import ipywidgets as widgets
from IPython.display import display

# Sliders para parámetros clave
beta_X_slider = widgets.FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='β_X')
alpha_X_slider = widgets.FloatSlider(value=0.1, min=0.05, max=0.3, step=0.01, description='α_X')
beta_Y_slider = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.1, description='β_Y')
alpha_Y_slider = widgets.FloatSlider(value=0.05, min=0.02, max=0.2, step=0.01, description='α_Y')

def simulate_delay(beta_X, alpha_X, beta_Y, alpha_Y):
    # Configuración base
    params_ffl, t_span, y0_ffl = build_config(model="ffl", input_type="step", t_span=(0, 100))
    params_cascade, _, y0_cascade = build_config(model="cascade", input_type="step", t_span=(0, 100))

    # Ajustar parámetros según sliders
    params_ffl["X"]["beta"] = beta_X
    params_ffl["X"]["alpha"] = alpha_X
    params_ffl["Y"]["beta"] = beta_Y
    params_ffl["Y"]["alpha"] = alpha_Y

    params_cascade["X"]["beta"] = beta_X
    params_cascade["X"]["alpha"] = alpha_X

    # Simulación
    t_ffl, y_ffl = run_simulation(model_ffl_and, params_ffl, t_span, y0_ffl)
    t_cascade, y_cascade = run_simulation(model_cascade, params_cascade, t_span, y0_cascade)

    # Visualización
    plot_timeseries(t_ffl, y_ffl, labels=["X", "Y", "Z (FFL)"])
    plot_timeseries(t_cascade, y_cascade, labels=["X", "Z (simple)"])

    # Análisis
    t_act_ffl = compute_activation_time(t_ffl, y_ffl[-1])
    t_act_cascade = compute_activation_time(t_cascade, y_cascade[-1])
    print(f"Tiempo activación Z (FFL): {t_act_ffl:.2f}")
    print(f"Tiempo activación Z (simple): {t_act_cascade:.2f}")

# Crear interfaz interactiva
widgets.interactive(simulate_delay, beta_X=beta_X_slider, alpha_X=alpha_X_slider,
                    beta_Y=beta_Y_slider, alpha_Y=alpha_Y_slider)


### **Ejercicio 2: Respuesta a Pulsos y Filtrado Temporal**

**Objetivo:**  
Explorar cómo el motivo **FFL AND con cooperatividad** actúa como **filtro pasa-largos**, ignorando señales breves y respondiendo solo a pulsos persistentes.

**Instrucciones:**

1.  Usa los **sliders interactivos** para modificar:
    *   **Duración del pulso** (en unidades de tiempo).
    *   **β\_Y** y **α\_Y** (síntesis y degradación de Y).
    *   **K_yz** (umbral para Y en la activación de Z).
    *   **nY** (exponente de Hill para cooperatividad en Y→Z).
2.  Observa:
    *   ¿Qué ocurre con Z cuando el pulso es muy corto?
    *   ¿Qué parámetros hacen que el filtro sea más estricto?
3.  Analiza los valores impresos:
    *   Máximo de Z.
    *   Tiempo de activación (si alcanza el 50% del máximo).
4.  **Pregunta clave:** ¿Cuál es la duración mínima del pulso para que Z se active significativamente?

**Concepto clave:**  
El FFL AND con cooperatividad introduce un **umbral temporal**: solo señales persistentes activan Z, lo que evita respuestas espurias ante ruido.

In [ ]:
#@title Ejercicio 2: Respuesta a Pulsos (Interactivo)

import ipywidgets as widgets
from IPython.display import display

# Sliders para parámetros clave
dur_slider = widgets.IntSlider(value=5, min=1, max=40, step=1, description='Duración pulso')
beta_Y_slider = widgets.FloatSlider(value=0.4, min=0.1, max=1.0, step=0.05, description='β_Y')
alpha_Y_slider = widgets.FloatSlider(value=0.10, min=0.05, max=0.3, step=0.01, description='α_Y')
Kp_slider = widgets.FloatSlider(value=2.5, min=1.0, max=10.0, step=0.5, description='K_yz')
nY_slider = widgets.IntSlider(value=3, min=1, max=4, step=1, description='nY')

# Función auxiliar para generar pulso
def make_pulse(t_on=0.0, duration=5.0, amplitude=1.0):
    return lambda t: amplitude if (t >= t_on) and (t <= t_on + duration) else 0.0

# Modelo con cooperatividad
def model_ffl_and_hill(t, y, params):
    X, Y, Z = y
    u = params["input"](t) # Corrected: Call the input function with time t
    beta_X, alpha_X = params["X"]["beta"], params["X"]["alpha"]
    beta_Y, alpha_Y, K_Y = params["Y"]["beta"], params["Y"]["alpha"], params["Y"]["K"]
    beta_Z, alpha_Z, K_Z, Kp_Z = params["Z"]["beta"], params["Z"]["alpha"], params["Z"]["K"], params["Z"]["Kp"]
    nY = params["Z"].get("nY", 3)
    dXdt = beta_X * u - alpha_X * X
    dYdt = (beta_Y * X / (K_Y + X)) - alpha_Y * Y
    act_X = X / (K_Z + X)
    act_Y = (Y**nY) / (Kp_Z**nY + Y**nY)
    dZdt = beta_Z * act_X * act_Y - alpha_Z * Z
    return [dXdt, dYdt, dZdt]

# Función interactiva
def simulate_pulse(duration, beta_Y, alpha_Y, Kp_Z, nY):
    # Configuración base
    params, t_span, y0 = build_config(model="ffl", input_type="step", t_span=(0, 100))
    params["input"] = make_pulse(t_on=0.0, duration=duration, amplitude=1.0)

    # Ajustes
    params["X"]["beta"] = 1.0
    params["X"]["alpha"] = 0.2
    params["Y"]["beta"] = beta_Y
    params["Y"]["alpha"] = alpha_Y
    params["Y"]["K"] = 1.0
    params["Z"]["beta"] = 2.0
    params["Z"]["alpha"] = 0.08
    params["Z"]["K"] = 1.5
    params["Z"]["Kp"] = Kp_Z
    params["Z"]["nY"] = nY

    # Simulación
    t, y = run_simulation(model_ffl_and_hill, params, t_span, y0)

    # Visualización
    plot_timeseries(t, y, labels=["X", "Y", "Z"])

    # Análisis
    max_Z = np.max(y[-1])
    t_act = compute_activation_time(t, y[-1], threshold=0.5)
    print(f"Duración pulso: {duration}")
    print(f"Máximo Z: {max_Z:.3f}")
    print(f"t_activación Z (50% máx): {t_act if t_act is not None else 'No activa'}")

# Interfaz interactiva
widgets.interactive(simulate_pulse,
                    duration=dur_slider,
                    beta_Y=beta_Y_slider,
                    alpha_Y=alpha_Y_slider,
                    Kp_Z=Kp_slider,
                    nY=nY_slider)





### **Ejercicio 3: Barrido de Parámetros y Retardo en la Activación**

**Objetivo:**  
Explorar cómo el **retardo en la activación de Z** depende de los parámetros del motivo FFL AND, en particular:

*   **β\_Y** (tasa de síntesis de Y).
*   **α\_Y** (tasa de degradación de Y).

**Instrucciones:**

1.  Usa los **controles interactivos** para definir los rangos de barrido:
    *   Valores mínimos y máximos para **β\_Y** y **α\_Y**.
    *   Número de pasos en cada rango.
2.  Haz clic en el botón para ejecutar el barrido y generar un **mapa de calor**:
    *   El color indica el **tiempo de activación de Z** (tiempo para alcanzar el 50% del máximo).
3.  Analiza:
    *   ¿Qué combinación de parámetros produce el mayor retardo?
    *   ¿Qué combinación hace que Z se active más rápido?
4.  **Pregunta clave:** ¿Por qué el retardo depende tanto de la dinámica de Y?

**Concepto clave:**  
El FFL actúa como filtro temporal porque la acumulación de Y es necesaria para activar Z. Si Y se sintetiza lentamente o se degrada rápido, el retardo aumenta.




In [ ]:

#@title Ejercicio 3: Barrido de Parámetros (Interactivo)

import ipywidgets as widgets
from IPython.display import display
import seaborn as sns

# Sliders para definir rangos y resolución
beta_min_slider = widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description='β_Y min')
beta_max_slider = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='β_Y max')
alpha_min_slider = widgets.FloatSlider(value=0.02, min=0.01, max=0.2, step=0.01, description='α_Y min')
alpha_max_slider = widgets.FloatSlider(value=0.1, min=0.05, max=0.3, step=0.01, description='α_Y max')
steps_slider = widgets.IntSlider(value=6, min=3, max=15, step=1, description='Resolución')

# Botón para ejecutar el barrido
run_button = widgets.Button(description="Ejecutar barrido", button_style='success')

# Función para generar heatmap
def run_parameter_scan(b_min, b_max, a_min, a_max, steps):
    # Configuración base
    params_base, t_span, y0 = build_config(model="ffl", input_type="step", t_span=(0, 100))

    # Definir rangos
    beta_range = np.linspace(b_min, b_max, steps)
    alpha_range = np.linspace(a_min, a_max, steps)

    # Matriz de resultados
    results = np.zeros((len(alpha_range), len(beta_range)))

    for i, alpha_val in enumerate(alpha_range):
        for j, beta_val in enumerate(beta_range):
            params = {k: (v.copy() if isinstance(v, dict) else v) for k, v in params_base.items()}
            params["Y"]["beta"] = beta_val
            params["Y"]["alpha"] = alpha_val

            # Simulación
            t, y = run_simulation(model_ffl_and, params, t_span, y0)
            activation_time = compute_activation_time(t, y[-1], threshold=0.5)
            results[i, j] = activation_time if activation_time is not None else np.nan

    # Visualización con Seaborn
    plt.figure(figsize=(8, 6))
    ax = sns.heatmap(results, xticklabels=np.round(beta_range, 2), yticklabels=np.round(alpha_range, 2),
                     cmap="viridis", annot=False, cbar_kws={'label': 'Tiempo de activación'})
    ax.set_xlabel("β_Y")
    ax.set_ylabel("α_Y")
    plt.title("Mapa de calor: tiempo de activación de Z")
    plt.show()

# Conectar botón con función
def on_button_click(b):
    run_parameter_scan(beta_min_slider.value, beta_max_slider.value,
                       alpha_min_slider.value, alpha_max_slider.value,
                       steps_slider.value)

run_button.on_click(on_button_click)

# Mostrar interfaz
display(beta_min_slider, beta_max_slider, alpha_min_slider, alpha_max_slider, steps_slider, run_button)
